In [117]:
import numpy as np
from qiskit import *
import cv2
%matplotlib inline

def I(cim, N):
    c = np.zeros((N, 2), dtype=np.complex)
    theta = np.zeros(N)
    phi = np.zeros(N) # All 0 for now

    # Get array of thetas, from Eq 6.
    for k in range(N):
        theta[k] = (cim[k][0]*(2**16) + cim[k][1]*(2**8) + cim[k][2])*np.pi/((2**24)-1)

    for k in range(N):
        c[k] = [np.cos(theta[k]/2), np.exp(1j*phi[k])*np.sin(theta[k]/2)]
        
    # Define basis states; there are 2^(2n) of them
    basis_state = np.diag(np.ones(N))
    
    # Store the image as a state
    state = np.zeros(2*N)
    for k in range(N):
        state = state + np.kron(basis_state[k], c[k])
        
    return (1/(2**n))*state

n = 1
height = 2**n
width = 2**n
N = 2**(2*n)

cim = np.zeros((N, 3), dtype=np.uint8) # classical image stored as (1d) array of rgb values
cim = [[128, 0, 0], [0, 128, 0], [0, 0, 128], [128, 127, 128]] # Input image

In [118]:
quantum_image = I(cim, N)
# print(cim)
# print(quantum_image)

In [119]:
theta_out = np.zeros(N)
basis_state = np.diag(np.ones(N))
rgb_out = np.zeros((N, 3), dtype=np.uint8)

for k in range(N):
    # Implementing equation 61
    p1 = np.outer(np.kron(basis_state[k], [0, 1]),np.kron(basis_state[k],[0, 1]))
    pm1 = np.real(quantum_image.T.conj() @ p1 @ quantum_image)
    # print(pm1)
    theta_out[k] = 2*np.arcsin(np.sqrt((2**(2*n))*pm1))
    
    # eq 62-64
    cr = np.int((256*256*256 - 1)*theta_out[k]/(256*256*np.pi))
    cg = np.int((((256*256*256 - 1)*theta_out[k]/np.pi) - cr*256*256)/256)
    cb = np.int(((256*256*256 - 1)*theta_out[k]/np.pi) - cr*256*256 - cg*256)
    
    rgb_out[k] = [np.int(cr), np.int(cg), np.int(cb)]
    
print(rgb_out) # Output should match input image further up the page

[[128   0   0]
 [  0 128   0]
 [  0   0 128]
 [128 127 128]]


In [19]:
# # Make images
# from PIL import Image

# for k in range(cim.shape[0]):
#     cim[k][0] = 255*(k%2)
# img = Image.fromarray(cim)
# img.save('testrgb.png')

In [ ]:
# from itertools import product

# def kron(v):
#     length = v.shape[0]
#     if length == 1:
#         return v[0]
#     else:
#         a0 = v[0]        
#     for i in range(1, length):
#         a0 = np.kron(a0, v[i])
#     return a0

# basis_state_temp = np.array(list(product([[1,0], [0,1]], repeat=2*n)))